In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW

In [ ]:
def load_data(file_path):
    """Load data from the Spider dataset JSON file."""
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def preprocess_data(data, tokenizer):
    """Preprocess data by tokenizing questions and SQL queries."""
    input_ids = []
    attention_masks = []
    labels = []

    for item in data:
        # Consider including db_id or schema information if needed
        question = item['question']
        sql_query = item['query']

        # Prepend the task description
        input_text = f"translate English to SQL: {question}"
        target_text = sql_query

        # Tokenize inputs and targets
        input_encoded = tokenizer(input_text, max_length=512, truncation=True, padding='max_length', return_tensors='pt')
        target_encoded = tokenizer(target_text, max_length=512, truncation=True, padding='max_length', return_tensors='pt')

        input_ids.append(input_encoded['input_ids'].squeeze())
        attention_masks.append(input_encoded['attention_mask'].squeeze())
        labels.append(target_encoded['input_ids'].squeeze())

    # Convert lists to tensors
    input_ids = torch.stack(input_ids)
    attention_masks = torch.stack(attention_masks)
    labels = torch.stack(labels)

    return input_ids, attention_masks, labels

def setup_t5_model(model_name='t5-small'):
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    return tokenizer, model

tokenizer, model = setup_t5_model()

# Load and preprocess Spider dataset
spider_train_data = load_data('/content/drive/MyDrive/ Text to Sql/spider/train_spider.json')  # Adjust path as necessary


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
input_ids, attention_masks, labels = preprocess_data(spider_train_data, tokenizer)

# Create TensorDataset and DataLoader
train_dataset = TensorDataset(input_ids, attention_masks, labels)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Setup the optimizer and training environment
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
# Training loop
model.train()
num_epochs = 3
for epoch in range(num_epochs):
    i=0
    total_loss = 0
    for batch in train_loader:
        i=i+1
        print(i)
        optimizer.zero_grad()
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")
    print(f"There were {i} iterations")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
model.save_pretrained('/content/drive/MyDrive/Saved Model/my_t5_sql_model')
tokenizer.save_pretrained('/content/drive/MyDrive/Saved Model/my_t5_sql_tokenizer')


In [ ]:
def evaluate_model(model, data_loader, device):
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    predictions = []
    actuals = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

            # Collect loss
            loss = outputs.loss
            total_loss += loss.item()

            # Decode predictions
            predicted_sql = tokenizer.batch_decode(outputs.logits.argmax(dim=-1), skip_special_tokens=True)
            actual_sql = tokenizer.batch_decode(labels, skip_special_tokens=True)

            predictions.extend(predicted_sql)
            actuals.extend(actual_sql)

    avg_loss = total_loss / len(data_loader)
    return avg_loss, predictions, actuals

In [ ]:
model.save_pretrained('/content/drive/MyDrive/Saved_models/my_t5_sql_model')
tokenizer.save_pretrained('/content/drive/MyDrive/Saved_models/my_t5_sql_model')


('/content/drive/MyDrive/Saved_models/my_t5_sql_model/tokenizer_config.json',
 '/content/drive/MyDrive/Saved_models/my_t5_sql_model/special_tokens_map.json',
 '/content/drive/MyDrive/Saved_models/my_t5_sql_model/spiece.model',
 '/content/drive/MyDrive/Saved_models/my_t5_sql_model/added_tokens.json')

In [ ]:

spider_eval_data = load_data('/content/drive/MyDrive/ Text to Sql/spider/train_others.json')  # Adjust path as necessary

# Assume spider_eval_data is loaded and preprocessed similarly to spider_train_data
input_ids_eval, attention_masks_eval, labels_eval = preprocess_data(spider_eval_data, tokenizer)
eval_dataset = TensorDataset(input_ids_eval, attention_masks_eval, labels_eval)
eval_loader = DataLoader(eval_dataset, batch_size=2, shuffle=False)

# Evaluate the model
avg_loss, predictions, actuals = evaluate_model(model, eval_loader, device)
print(f"Average Loss: {avg_loss}")

# Optionally, you can compute more sophisticated metrics like exact match, etc.
from sklearn.metrics import accuracy_score

# Simple accuracy (exact match might require more complex comparison)
accuracy = accuracy_score(actuals, predictions)
print(f"Accuracy: {accuracy}")


NameError: name 'load_data' is not defined